In [1]:
import pygplates
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr


import paleogeography as pg
reload(pg)

%matplotlib inline


#####################################
basedir = '/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles/'

time = 255.

sampling = 0.5

rotation_model = pygplates.RotationModel(['%s/Global_EB_250-0Ma_GK07_Matthews++.rot' % basedir,
                                          '%s/Global_EB_410-250Ma_GK07_Matthews++.rot' % basedir])

pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,time)


#####################################
MODELDIR = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/'
RotFile_List = ['%s/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot' % MODELDIR]
GPML_List = ['%s/DynamicPolygons/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_TopologyBuildingBlocks.gpmlz' % MODELDIR]
#####################################
rotation_model = pygplates.RotationModel(RotFile_List)
topology_features = pygplates.FeatureCollection()
for file in GPML_List:
    topology_feature = pygplates.FeatureCollection(file)
    topology_features.add(topology_feature)

resolved_topologies = []
shared_boundary_sections = []
pygplates.resolve_topologies(topology_features, rotation_model, resolved_topologies, time, shared_boundary_sections)
##################################### 



In [2]:
# load all the features from the different environment categories into a single collection of features
pg_features = pg.load_paleogeography(pg_dir)

# rasterise points using default environment list - result is a set of multipoints
pg_points = pg.rasterise_paleogeography(pg_features,rotation_model,time,sampling=sampling)



['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_255Ma/lm_fig32_269_248_PresentDay_Paleogeog_Matthews2016_255.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_255Ma/m_fig32_269_248_PresentDay_Paleogeog_Matthews2016_255.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_255Ma/sm_fig32_269_248_PresentDay_Paleogeog_Matthews2016_255.00Ma.shp']
[]
no features of type i


In [ ]:
deep_ocean_features = []
for feature in pg_points:
    if feature.get_shapefile_attribute('Layer') is None:
        deep_ocean_features.append(feature)

pdepth_points = pg.paleobathymetry_from_topologies(resolved_topologies,
                                                   shared_boundary_sections,
                                                   deep_ocean_features)


Generating distances for Plate 430 ...
Generating distances for Plate 9010 ...
Generating distances for Plate 9009 ...
Generating distances for Plate 604 ...
Generating distances for Plate 902009 ...
Generating distances for Plate 902013 ...
Generating distances for Plate 9014 ...
Generating distances for Plate 601 ...
Generating distances for Plate 602 ...
Generating distances for Plate 701 ...
Generating distances for Plate 580 ...
Generating distances for Plate 505 ...
Generating distances for Plate 926 ...
Generating distances for Plate 902 ...
Generating distances for Plate 16111 ...
Generating distances for Plate 6022 ...
Generating distances for Plate 919 ...
Generating distances for Plate 176 ...


In [ ]:
env_color_dict = {'lm':'green',
                  'm':'orange',
                  'sm':'lightblue',
                  'i':'blue'}

plt.figure(figsize=(15,8))

m = pg.paleogeography_points_basemap(pg_points,env_color_dict,fill_color='darkblue',markersize=3)

plt.show()


In [ ]:
topo_dict = {'lm':400.,
             'm':3000.,
             'sm':-200.,
             'i':500.}

# create a grid from the paleogeography points, mapping height values to each
# point based on the environment and the dictionary specified above

bathymetry_mode = 'fill'

if bathymetry_mode is 'fill':
    # Fill all values in deep ocean with constant
    pg.paleogeography2topography_xyz(pg_points,topo_dict,sampling,
                                     fill_value=-3000,grdfile='test.nc')

elif bathymetry_mode is 'infer':
    # Fill all values in deep ocean with proxy depth from distance to ridges 
    pg.paleogeography2topography_xyz(pg_points,topo_dict,sampling,
                                     bathymetry_points=pdepth_points,grdfile='test.nc')


# optinally perform some smoothing to remove sharp discontinuities at each 
# environment boundary
pg.smooth_topography_grid('test.nc','test_filt.nc',1000.)


gridX,gridY,gridZ = pg.load_netcdf('test_filt.nc')
#gridX,gridY,gridZ = pg.load_netcdf('test.nc')


plt.figure(figsize=(20,8))
plt.contourf(gridX,gridY,gridZ,
             levels=np.hstack((np.arange(-6000,1,400),np.arange(200,3001,200))),
             cmap=plt.cm.BrBG_r)
plt.gca().set_aspect('equal')
plt.colorbar()
plt.show()


In [ ]:
# Specify points for profile

#PtLons = [-86,-35]
#PtLats = [0,0] 

PtLons = [140,130]
PtLats = [-40,65]

# derive a great-circle profile lines from the anchor points
GreatCirclePoints,ProfilePoints,arc_distance = pg.create_profile_points(PtLons,PtLats)

# create an array of distances along profile in km, starting at zero
ProfileX_kms = np.arange(0,ProfilePoints.shape[0])*arc_distance

# extract the values from the (smoothed) topography grid along the profile
topo_profile = pg.create_slice(gridX,gridY,gridZ,GreatCirclePoints,ProfilePoints)

plt.plot(topo_profile)


In [ ]:
# generate a pygplates polyline feature from the densely sampled great-circle points
cross_section_geometry = pygplates.PolylineOnSphere(GreatCirclePoints)

# Find where the profile line intersects plate boundaries from the resolved topologies
# [TODO - maybe would be better to use the untesselated line segments for this??]
(subduction_intersections,
 ridge_intersections,
 other_intersections) = pg.plate_boundary_intersections(cross_section_geometry,shared_boundary_sections,ProfileX_kms)

print 'Profile crosses: \n%d subduction zones \n%d Mid-ocean ridges \n%d other boundaries' \
    % (len(subduction_intersections), len(ridge_intersections), len(other_intersections))


In [ ]:

# Get Moho from topography assuming isostasy
moho_profile = pg.topo2moho(topo_profile,ref_depth=22000)

# plot the profile
plt.figure(figsize=(25,3))
pg.paleogeography_cross_section(ProfileX_kms,topo_profile,moho_profile,
                                subduction_intersections,ridge_intersections,
                                vertical_exaggeration=10.)
plt.show()



plt.figure(figsize=(20,8))
plt.contourf(gridX,gridY,gridZ,np.arange(-3000,3001,200),cmap=plt.cm.BrBG_r)
plt.gca().set_aspect('equal')
plt.plot(GreatCirclePoints[:,1],GreatCirclePoints[:,0],'k',linewidth=3)
for point in subduction_intersections:
    plt.plot(point[0].to_lat_lon()[1],point[0].to_lat_lon()[0],'ro')
for point in ridge_intersections:
    plt.plot(point[0].to_lat_lon()[1],point[0].to_lat_lon()[0],'yo')
for point in other_intersections:
    plt.plot(point[0].to_lat_lon()[1],point[0].to_lat_lon()[0],'go')
plt.show()



In [ ]:
print subduction_intersections


In [ ]:
from reconstruction_classes import *
from raster_reconstruction_classes import *
#gridX,gridY,gridZ = pg.load_netcdf('/Users/simon/GIT/agegrid-dev/M16a/masked/M16_seafloor_age_mask_{:0.1f}Ma.nc'.format(time))
seafloor_age_grid = GplatesRaster('/Users/simon/GIT/agegrid-dev/M16a/masked/M16_seafloor_age_mask_{:0.1f}Ma.nc'.format(time))

#seafloor_age_profile = pg.create_slice(gridX,gridY,gridZ,GreatCirclePoints,ProfilePoints)

#print GreatCirclePoints

seafloor_age_profile = seafloor_age_grid.sample_using_gmt(GreatCirclePoints[:,1],GreatCirclePoints[:,0])

seafloor_depth_profile = pg.age2depth(seafloor_age_profile)/1000
seafloor_litho_thickness = np.sqrt(seafloor_age_profile)

profile_plate_ids = pg.profile_plate_ids(resolved_topologies,rotation_model,GreatCirclePoints)

daspect = 50

plt.figure(figsize=(20,2))
plt.plot(ProfileX_kms,seafloor_depth_profile, color='royalblue')
plt.plot(ProfileX_kms,seafloor_depth_profile-6., color='red')
#plt.plot(ProfileX_kms,seafloor_depth_profile-seafloor_litho_thickness*100)


for point in subduction_intersections:
    plt.plot(point[1],-5, 'ko', zorder=2, fillstyle='none')
    
    if point[2]:
        plt.plot([point[1],(point[1]-(10*daspect))],[-5,-30],linewidth=1,color='royalblue',zorder=1)
        plt.plot([(point[1]+(2*daspect)),(point[1]-(10*daspect)+(2*daspect))],[-11,-36],linewidth=1,color='red',zorder=1)
    else:
        plt.plot([point[1],(point[1])+(10*daspect)],[-5,-30],linewidth=1,color='royalblue',zorder=1)
        plt.plot([(point[1]-(2*daspect)),(point[1])+(10*daspect)-(2*daspect)],[-11,-36],linewidth=1,color='red',zorder=1)


    
plt.ylim(-40,5)
plt.xlim((ProfileX_kms.min(),ProfileX_kms.max()))
plt.gca().set_aspect(daspect)
plt.show()


plt.figure(figsize=(20,2))
plt.plot(ProfileX_kms,profile_plate_ids)
plt.scatter(ProfileX_kms,seafloor_depth_profile,c=seafloor_age_profile,cmap=plt.cm.gnuplot_r,vmin=0,vmax=120)
#plt.xlim((ProfileX_kms.min(),ProfileX_kms.max()))



In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def iterative_smoothing(seafloor_depth_profile,n_iter=5):
    
    is_ocean_depth = np.copy(seafloor_depth_profile)
    for i in range(n_iter):
        #print i
        seafloor_depth_profile[np.isnan(seafloor_depth_profile)] = 2.0
        seafloor_depth_profile = smooth(seafloor_depth_profile,3)
        seafloor_depth_profile[~np.isnan(is_ocean_depth)] = is_ocean_depth[~np.isnan(is_ocean_depth)]
        
    return seafloor_depth_profile
    
    
    

seafloor_depth_profile = pg.age2depth(seafloor_age_profile)/1000

#print seafloor_depth_profile    

for point in subduction_intersections:
    temp_array = np.array(point[1] - ProfileX_kms)
    if point[2]:
        temp_array = temp_array*-1.
    temp_array[temp_array<0] = 9999999
    
    index_of_trench = temp_array.argmin()
    #seafloor_depth_profile[index_of_trench] = -6.
    
    subducting_plate = profile_plate_ids[index_of_trench]
    
    # get an array with the depths just within one plate polygon
    depths_in_plate = seafloor_depth_profile[np.equal(profile_plate_ids,subducting_plate)]
    
    #print depths_in_plate, depths_in_plate.shape        
    if ~point[2]:
        depths_in_plate = np.flip(depths_in_plate) #[-1:0:-1]
    #print depths_in_plate
    if np.any(np.isnan(depths_in_plate)):
        index = np.where(~np.isnan(depths_in_plate))[0].min()
        #print index_of_trench, index, seafloor_depth_profile[index_of_trench+index+1]
        
        #print index
    
        # NEED TO FIGURE OUT FORWARDS REVERSE TO INDEX RESULT BACK INTO FULL ARRAY
        if ~point[2]:
            seafloor_depth_profile[index_of_trench-index:index_of_trench+1] = depths_in_plate[index]
            #print seafloor_depth_profile[index_of_trench-1:index_of_trench+2]
            #print seafloor_depth_profile[index_of_trench]
            #print seafloor_depth_profile[index_of_trench-index-1]
            #seafloor_depth_profile[index_of_trench] = seafloor_depth_profile[index_of_trench+1]
        else:
            seafloor_depth_profile[index_of_trench:index_of_trench+index] = depths_in_plate[index]
            #seafloor_depth_profile[index_of_trench-1] = seafloor_depth_profile[index_of_trench]
    #if
    
    
    #testing - also make first point on othetr side of the trench the same depth
    #seafloor_depth_profile
    
#is_ocean_depth = seafloor_depth_profile
#seafloor_depth_profile[np.isnan(seafloor_depth_profile)] = 1.
    
#print seafloor_depth_profile

land_ocean_index = ~np.isnan(seafloor_depth_profile)

#print land_ocean_index

res = iterative_smoothing(seafloor_depth_profile,n_iter=20)
moho = pg.topo2moho(res*1000,ref_depth=22000, rhoC=2200)/1000   

moho[land_ocean_index] = seafloor_depth_profile[land_ocean_index]-6
  
    
xgrid = np.vstack((ProfileX_kms,ProfileX_kms))
zgrid = np.vstack((res,moho))
tmp=np.vstack((seafloor_age_profile,seafloor_age_profile))
    
plt.figure(figsize=(20,10))
#plt.plot(ProfileX_kms,seafloor_depth_profile, '.-', color='royalblue')
#plt.plot(ProfileX_kms,seafloor_depth_profile-6., color='red')

#plt.plot(ProfileX_kms,seafloor_depth_profile-seafloor_litho_thickness*100)

plt.plot(ProfileX_kms,res, color='grey')
#plt.plot(ProfileX_kms,moho,'k')

plt.fill_between(ProfileX_kms,res,moho,color='grey')

for point in subduction_intersections:
    #plt.plot(point[1],-5, 'ko', zorder=2, fillstyle='none')
    
    if point[2]:
        slab_top_X = np.array([point[1],(point[1]-(10*daspect))])
        slab_top_Y = np.array([-5,-30])
        slab_base_X = slab_top_X+150
        slab_base_Y = slab_top_Y-6
    else:
        slab_top_X = np.array([point[1],(point[1]+(10*daspect))])
        slab_top_Y = np.array([-5,-30])
        slab_base_X = slab_top_X-150
        slab_base_Y = slab_top_Y-6
    #plt.plot(slab_top_X,slab_top_Y,linewidth=1,color='k',zorder=1)
    plt.plot(slab_top_X,slab_top_Y,linewidth=1,color='k',zorder=1)
    #plt.fill_between(slab_top_X,slab_top_Y,slab_top_Y-6,color='black',zorder=5)
    plt.fill_betweenx(slab_top_Y,slab_top_X,slab_base_X,color='black',zorder=5)
        
    #else:
    #    plt.plot([point[1],(point[1])+(10*daspect)],[-5,-30],linewidth=1,color='royalblue',zorder=1)
    #    plt.plot([point[1],(point[1])+(10*daspect)],[-16,-41],linewidth=1,color='red',zorder=1)

plt.pcolormesh(xgrid,zgrid,tmp,cmap=plt.cm.inferno_r,vmin=-10,vmax=250)
#plt.pcolormesh(xgrid,zgrid,tmp,cmap=plt.cm.inferno_r,vmin=-10,vmax=0)
   
plt.ylim(-50,5)
plt.xlim((ProfileX_kms.min(),ProfileX_kms.max()))
plt.gca().set_aspect(daspect)
plt.show()

